In [1]:
import pandas as pd
import os

In [2]:
# Ottiene la directory di lavoro corrente
script_dir = os.getcwd()

# Costruisce il percorso relativo al file CSV
file_path = os.path.join(script_dir, "Data", "main_data", "releases.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "Data", "clean_data")

# Crea la cartella clean_data se non esite
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)

In [3]:
# Rimuove duplicati
df.drop_duplicates(inplace=True)

In [4]:
# Rimuove righe con troppi valori nulli (ad esempio, più del 50% dei dati mancanti)
df.dropna(thresh=df.shape[1] * 0.5, inplace=True)

# Sostituisce i valori nulli con "Unknown"
df['country'] = df['country'].fillna("Unknown")
df['date'] = df['date'].fillna("Unknown")
df['type'] = df['type'].fillna("Unknown")
df['rating'] = df['rating'].fillna("Unknown")

In [5]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "releases_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

Caricamento file csv pulito nel database con MongoDB


In [6]:
from pymongo import MongoClient

In [7]:
# Configura MongoDB (modifica se necessario)
MONGO_URL = "mongodb://localhost:27017/"
DB_NAME = "ium_database"
COLLECTION_NAME = "releases_data"

# Connessione a MongoDB
client = MongoClient(MONGO_URL)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

# Carica il CSV con Pandas
df = pd.read_csv(cleaned_csv_path)

# Converte il DataFrame in JSON-like per MongoDB
data = df.to_dict(orient="records")

# Usa insert_many() per inserire tutto in un'unica operazione (molto più veloce)
collection.insert_many(data, ordered=False); #il ; non fa stampare l'output su jupyter

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67b9a4557b1d0acc7b165b99, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>